In [2]:
import numpy as np
import scipy as scp

In [43]:
np.random.uniform(size=(10, 10))

array([[0.42646838, 0.03812715, 0.88597842, 0.99294523, 0.014166  ,
        0.79314199, 0.46882595, 0.29673332, 0.86057824, 0.81801752],
       [0.16732116, 0.90519451, 0.3887167 , 0.87802433, 0.66312936,
        0.41093569, 0.91661174, 0.10934841, 0.38533521, 0.8479159 ],
       [0.44641189, 0.87649086, 0.25604911, 0.69222683, 0.27673748,
        0.06972736, 0.51618033, 0.88290112, 0.59156674, 0.94927261],
       [0.15703096, 0.83965002, 0.18195224, 0.43250677, 0.1521192 ,
        0.25980485, 0.27545836, 0.76086931, 0.26918801, 0.13471756],
       [0.44464649, 0.1131265 , 0.44535106, 0.52211015, 0.85920107,
        0.69036088, 0.87082674, 0.82465213, 0.18350347, 0.36813342],
       [0.98157423, 0.87931151, 0.97134457, 0.4593007 , 0.41223666,
        0.58904006, 0.13712508, 0.63596533, 0.73450552, 0.13534961],
       [0.29665975, 0.45247505, 0.1984255 , 0.69683371, 0.77182687,
        0.99443565, 0.15739575, 0.46258091, 0.14583192, 0.13806172],
       [0.28217884, 0.25410107, 0.5943203

In [2]:
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.stats import multivariate_normal, invwishart
def random_data_gen(n_samples=1000, n_feats=10, maha=1.0, ratio=0.5, seed=None):
    if seed:
        np.random.seed(seed)
    ## initialize multivariate normal dist with normally distributed means and covariance
    ## drawn from an inverse wishart distribution (conjugate prior for MVN)
    norm_means_a = np.random.randn(n_feats)
    norm_means_b = np.zeros_like(norm_means_a)
    wishart_cov = invwishart(n_feats+1, np.identity(n_feats)).rvs()
    dist = mahalanobis(norm_means_a, norm_means_b, wishart_cov)
    norm_means_a = norm_means_a * (maha / dist)
    assert np.isclose(mahalanobis(norm_means_a, norm_means_b, wishart_cov), maha)
    ## multivariate normal distributions with different means and equal variances
    corr = (D:=np.diag(1/np.sqrt(np.diag(wishart_cov)))) @ wishart_cov @ D
    print(corr)
    mvn_a = multivariate_normal(mean=norm_means_a, cov=wishart_cov)
    mvn_b = multivariate_normal(mean=norm_means_b, cov=wishart_cov)
    ## generate data samples from a multivariate normal
    data = np.vstack([mvn_a.rvs(int(n_samples*ratio)), mvn_b.rvs(n_samples - int(n_samples*ratio))])
    labels = np.arange(len(data))<int(n_samples*ratio)
    return data, labels

/home1/jrudoler/anaconda3/envs/py38/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data, labels = random_data_gen(10000, 10, 1., ratio=.5)
data, labels

[[ 1.          0.93630848  0.74308037 -0.54232023  0.03742594 -0.82100354
  -0.55853534  0.24242218  0.87592011 -0.95996785]
 [ 0.93630848  1.          0.63964175 -0.53283238  0.07583815 -0.70510927
  -0.58046352  0.06159404  0.89608207 -0.94588717]
 [ 0.74308037  0.63964175  1.         -0.20331291 -0.13037073 -0.66388417
  -0.34421476  0.305216    0.6764147  -0.76198328]
 [-0.54232023 -0.53283238 -0.20331291  1.          0.06514315  0.33054814
   0.28391272 -0.28666711 -0.6102961   0.48294586]
 [ 0.03742594  0.07583815 -0.13037073  0.06514315  1.          0.29622569
   0.16467777  0.09439198 -0.05227906 -0.00448569]
 [-0.82100354 -0.70510927 -0.66388417  0.33054814  0.29622569  1.
   0.64063703 -0.10255239 -0.64173294  0.75378777]
 [-0.55853534 -0.58046352 -0.34421476  0.28391272  0.16467777  0.64063703
   1.          0.13599285 -0.53737821  0.44166916]
 [ 0.24242218  0.06159404  0.305216   -0.28666711  0.09439198 -0.10255239
   0.13599285  1.          0.12912509 -0.14998772]
 [ 0.875

(array([[ 1.18061685,  1.40038676,  0.14969712, ...,  0.22890684,
          0.37973539, -1.6533282 ],
        [-1.74391233,  0.12293226, -0.12762751, ..., -0.13437358,
         -1.19559448, -0.01445753],
        [ 2.41552433,  2.23637525, -0.3910104 , ..., -0.52361656,
         -0.4152034 , -3.09471946],
        ...,
        [-2.8175577 , -1.18083928, -1.47259052, ..., -0.05725054,
         -0.52044143,  2.37584459],
        [ 0.21536665,  0.04750757,  0.51846437, ...,  0.9291054 ,
         -0.05429538, -0.24823403],
        [-3.75664041, -0.93466316, -1.07942532, ..., -0.70898992,
         -0.97120897,  2.40679969]]),
 array([ True,  True,  True, ..., False, False, False]))

In [80]:
np.save("random_data_X.npy", data)
np.save("random_data_y", labels)

In [3]:
np.random.randn?

Docstring:
randn(d0, d1, ..., dn)

Return a sample (or samples) from the "standard normal" distribution.

.. note::
    This is a convenience function for users porting code from Matlab,
    and wraps `standard_normal`. That function takes a
    tuple to specify the size of the output, which is consistent with
    other NumPy functions like `numpy.zeros` and `numpy.ones`.

.. note::
    New code should use the ``standard_normal`` method of a ``default_rng()``
    instance instead; please see the :ref:`random-quick-start`.

If positive int_like arguments are provided, `randn` generates an array
of shape ``(d0, d1, ..., dn)``, filled
with random floats sampled from a univariate "normal" (Gaussian)
distribution of mean 0 and variance 1. A single float randomly sampled
from the distribution is returned if no argument is provided.

Parameters
----------
d0, d1, ..., dn : int, optional
    The dimensions of the returned array, must be non-negative.
    If no argument is given a single Python